In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Import Library

In [2]:
import os
import random
from collections import defaultdict

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import warnings

warnings.filterwarnings(action = "ignore")

# 2. Config Setup

In [3]:
def fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True


config = {"data_path":"/content/drive/MyDrive/23Inha_AI_RecSys/data",
          "model_path":"/content/drive/MyDrive/23Inha_AI_RecSys/model/model.pth",
          "submit_path":"/content/drive/MyDrive/23Inha_AI_RecSys/submit/submit_autorec.csv",
          "n_valid_sample":2, #검증 샘플 비율 or 개수
          "replace":True, #검증 샘플 복원 / 비복원 추출

          "epoch":30,
          "lr":5e-4,    # best | 5e-4 at epoch 13
          "batch_size":1024,
          "threshold":0., #평점 임계값

          "n_feature":128, # AutoRec 모델의 특성 차원 설정

          "top_k":50, # 추천 결과에서 상위 K개 아이템을 선택하기 위한 K 값 설정

          "seed":39,  # 재현성을 위한 난수 시드 설정
          "n_workers":2} # 데이터 로딩을 위한 병렬 작업자 수 설정

fix_seed(config["seed"])
device = "cuda" if torch.cuda.is_available() else "cpu"
os.makedirs(os.path.dirname(config["model_path"]), exist_ok = True)
os.makedirs(os.path.dirname(config["submit_path"]), exist_ok = True)

# 3. Data Preprocessing
- Rating 정보인 train.csv, Image / Review 정보인 image / text.npy, Label 정보인 user / item_label.npy로 구성
- image.npy에는 제품 별 Image를 CNN을 통해 가공한 Feature 제공
- text.npy에는 제품 별 Review를 Transformer를 통해 가공한 Feature 제공
- 단, image feature는 Memory Overflow 문제가 발생할 수 있어, 20000개 단위로 Split한 image_1-N.npy를 함께 제공

In [4]:
class Parser:
    def __init__(self, path, n_valid_sample = 1, threshold = 0., replace = False, seed = 0):
        """
                 user    item    interaction sparsity
        data     192,403 63,001  1,689,188   99.9861%

        rating scale - 1-5
        review feature shape - (I, 384)
        image feature shape - (I, 4096)
        """
        self.path = os.path.join(path, "train.csv")
        self.n_valid_sample = n_valid_sample
        self.threshold = threshold
        self.replace = replace
        self.seed = seed

        #rating > (N, [user id, item id, rating])
        self.raw_data = self.parse(self.path)
        self.tr_data, self.te_data = self.split_data(self.raw_data, n_sample = self.n_valid_sample, replace = self.replace, seed = self.seed)

        #tag > (item id, feature)
        #self.image = parse(os.path.join(path, "image.npy")) #image_1-N.npy > image.npy load시 memory overflow가 나는 경우를 위해, 20000개 단위 Split
        #self.text = parse(os.path.join(path, "text.npy"))

        self.user_label = np.load(os.path.join(path, "user_label.npy"))
        self.item_label = np.load(os.path.join(path, "item_label.npy"))
        self.user_encoder, self.user_decoder = self.generate_label_mapper(self.user_label)
        self.item_encoder, self.item_decoder = self.generate_label_mapper(self.item_label)
        self.tr_data = self.map_label(self.tr_data, self.user_encoder, self.item_encoder)
        self.te_data = self.map_label(self.te_data, self.user_encoder, self.item_encoder)

        self.tr_pos, self.tr_neg = self.generate_sequence_data(self.tr_data)
        self.te_pos, self.te_neg = self.generate_sequence_data(self.te_data, 3) #3점 이상인 경우 Positive

        self.exist_users = [i for i in range(len(self.user_label))]
        self.exist_items = [i for i in range(len(self.item_label))]

        self.random = np.random.RandomState(seed = self.seed)

    @staticmethod
    def parse(path):
        data = pd.read_csv(path).to_numpy()
        return data

    @staticmethod
    def split_data(data, n_sample = 1, replace = False, seed = None):
        """
        train / valid data로 split

        Args:
            data
        Returns:
            train data / valid data
        """
        users = defaultdict(list)
        ratings = defaultdict(list)
        user_train = []
        user_valid = []
        for d in data:
            users[int(d[0])].append(d[1])
            ratings[int(d[0])].append(d[2])

        random = (np.random.RandomState(seed) if seed is not None else np.random)
        for user in users:
            item, rating = users[user], ratings[user]
            n_valid_sample = max(int(np.round(len(item) * n_sample)) if isinstance(n_sample, float) else min(n_sample, len(item)), 1)
            n_valid_sample = (n_valid_sample - 1) if (len(item) == n_valid_sample) else n_valid_sample
            valid_indices = random.choice(len(item), size = n_valid_sample, replace = replace).tolist()
            valid = [item[i] for i in valid_indices]
            train = list(set(item) - set(valid))
            train_indices = [idx for idx, i in enumerate(item) if i in train]

            for i, r in zip(train, [rating[i] for i in train_indices]):
                user_train.append([user, i, r])

            for i, r in zip(valid, [rating[i] for i in valid_indices]):
                user_valid.append([user, i, r])
        user_train = np.array(user_train, dtype = data.dtype)
        user_valid = np.array(user_valid, dtype = data.dtype)
        return user_train, user_valid

    @staticmethod
    def generate_label_mapper(label):
        """
        mapper 생성

        Args:
            id list
        Returns:
            dict: 생성된 label encoder, decoder
        """
        encoder = {}
        decoder = {}
        for idx, _id in enumerate(label):
            encoder[_id] = idx
            decoder[idx] = _id
        return encoder, decoder

    @staticmethod
    def map_label(data, user_mapper, item_mapper):
        return np.array([[user_mapper[d[0]], item_mapper[d[1]], d[2]] for d in data], dtype = data.dtype)

    @staticmethod
    def generate_sequence_data(data, threshold = 0.):
        """
        sequence_data 생성

        Returns:
            dict: user 별 pos item sequence, neg item sequence
        """
        pos_users = defaultdict(list)
        neg_users = defaultdict(list)
        pos_ratings = defaultdict(list)
        neg_ratings = defaultdict(list)
        for d in data:
            if threshold <= d[2]: #설정한 임계 평점 값 기준으로 높으면 positive
                pos_users[int(d[0])].append(int(d[1]))
                neg_users[int(d[0])]
                pos_ratings[int(d[0])].append(d[2])
                neg_ratings[int(d[0])]
            else:  #설정한 임계 평점 값 기준으로 낮으면 negative
                pos_users[int(d[0])]
                neg_users[int(d[0])].append(int(d[1]))
                pos_ratings[int(d[0])]
                neg_ratings[int(d[0])].append(d[2])
        for user in list(pos_users.keys()):
            items = pos_users[user]
            pos_users[user] = [items[i] for i in np.argsort(pos_ratings[user])[::-1]]
            items = neg_users[user]
            neg_users[user] = [items[i] for i in np.argsort(neg_ratings[user])[::-1]]
        return pos_users, neg_users #각 유저 별 긍, 부정 아이템 시퀀스

    def neg_sampling(self, users, sampling_count = 4):
        """
        음성 샘플링을 수행하여 사용자와 함께 반환

        Args:
            users (numpy.ndarray): 사용자 배열
            sampling_count (int): 음성 샘플링 개수

        Returns:
            list, list: 음성 사용자 배열, 음성 아이템 배열
        """
        _users, neg_items = [], []
        for user in users:
            neg = self.tr_neg[user]
            neg = self.random.choice(neg, min(sampling_count, len(neg)), replace = False).tolist() if 0 < len(neg) else []
            if len(neg) < sampling_count:
                new_neg = list(set(self.exist_items) - set(self.tr_pos[user]))
                new_neg = self.random.choice(new_neg, sampling_count - len(neg), replace = False)
                neg = np.concatenate([neg, new_neg]).astype(np.int32).tolist()
            neg_items += neg
            _users += [user] * len(neg)
        return _users, neg_items

    @property
    def data(self):
        return self.tr_pos, self.te_pos

    def to_matrix(self, data, users):
        """
        autorec모델 학습을 위해 데이터를 행렬 형태로 변환.
        """
        mat = torch.zeros(users.size(0), len(self.item_label))
        for i, user in enumerate(users):
            j = data[user.item()]
            mat[i, j] = 1
        return mat

class Dataset(Dataset):
    def __init__(self, n_user):
        self.n_user = n_user
        self.users = [i for i in range(n_user)]

    def __len__(self):
        return self.n_user

    def __getitem__(self, i):
        return torch.LongTensor([self.users[i]])

parser = Parser(config["data_path"], config["n_valid_sample"], config["threshold"], config["replace"], config["seed"])

# 4. Define Model
- AutoRec(https://dl.acm.org/doi/10.1145/2740908.2742726)

In [6]:
class AutoRec(nn.Module):
    def __init__(self, n_item, n_feature, dropout=0.1):
        super(AutoRec, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(n_item, n_feature),  # 입력 크기인 n_item에서 n_feature로 이어지는 선형 레이어
            nn.Sigmoid(),
            #nn.Linear(n_feature, n_feature // 4),
        )
        self.decoder = nn.Sequential(
            #nn.Linear(n_feature // 4, n_feature),
            #nn.Identity(),
            nn.Linear(n_feature, n_item), # n_feature에서 n_item으로 이어지는 선형 레이어
            nn.Identity()
        )

        self.dropout = nn.Dropout(dropout)
        self.init_weights()

    def forward(self, mat):
        latent = self.dropout(self.encoder(mat)) # 입력 데이터를 인코더에 전달하여 잠재 특성을 얻음
        recont_mat = self.decoder(latent) # 잠재 특성을 디코더에 전달하여 재구성 행렬을 얻음
        return recont_mat

    def init_weights(self):
        """
        인코더와 디코더의 가중치를 초기화하는 함수.
        """
        for layer in self.encoder:
            if isinstance(layer, nn.Linear):
                size = layer.weight.size()
                fan_out = size[0]
                fan_in = size[1]
                std = np.sqrt(2.0/(fan_in + fan_out))
                layer.weight.data.normal_(0.0, std)
                layer.bias.data.normal_(0.0, 0.001)

        for layer in self.decoder:
            if isinstance(layer, nn.Linear):
                size = layer.weight.size()
                fan_out = size[0]
                fan_in = size[1]
                std = np.sqrt(2.0/(fan_in + fan_out))
                layer.weight.data.normal_(0.0, std)
                layer.bias.data.normal_(0.0, 0.001)

# 5. Experimental Setup

In [7]:
def ndcg(true_list, pred_list):  # NDCG (Normalized Discounted Cumulative Gain) 계산 함수
    true_list = true_list[:len(pred_list)]
    idcg = 1 / np.log2(np.arange(len(true_list)) + 2)
    dcg = 1 / np.log2([rank + 2 for rank, pred in enumerate(pred_list) if pred in true_list])
    return np.sum(dcg) / (np.sum(idcg) + 1e-12)

def train(model, data_loader, criterion, optimizer, parser, prefix = "", leave = True):
    prefix = prefix + " | " if len(prefix) != 0 else prefix

    model.train()
    loss_val = 0

    tbar = tqdm(data_loader, total = len(data_loader), leave = leave)
    try:
        for i, users in enumerate(tbar):
            mat = parser.to_matrix(tr_pos, users)
            mat = mat.to(device)
            recon_mat = model(mat)

            optimizer.zero_grad()
            loss = criterion(recon_mat, mat)

            loss = torch.sqrt(loss)     # *** modify HERE ***

            loss.backward()
            optimizer.step()

            loss_val += loss.item()
            tbar.set_description("{0}tr_loss: {1:.4e}".format(prefix, loss_val / (i + 1)))
    except Exception as e:
        raise e
    finally:
        tbar.close()
    loss_val /= len(data_loader)
    return loss_val

def evaluate(model, data_loader, parser, k = 50, prefix = "", leave = True):
    prefix = prefix + " | " if len(prefix) != 0 else prefix
    model.eval()

    NDCG = 0.0

    tr_pos, te_pos = parser.data

    tbar = tqdm(data_loader, total = len(data_loader), leave = leave)
    cnt = 0
    try:
        with torch.no_grad():
            for i, users in enumerate(tbar):
                mat = parser.to_matrix(tr_pos, users)
                mat = mat.to(device)

                recon_mat = model(mat)
                recon_mat = recon_mat.softmax(dim = 1)
                recon_mat[mat == 1] = -1.
                rec_list = recon_mat.argsort(dim = 1)

                for user, rec in zip(users, rec_list):
                    uv = te_pos[user.item()]
                    up = rec[-k:].cpu().numpy()[::-1].tolist()
                    NDCG += ndcg(true_list = uv, pred_list = up)
                    cnt += 1
                tbar.set_description("{0}ndcg: {1:.4f}".format(prefix, NDCG / cnt))
    except Exception as e:
        raise e
    finally:
        tbar.close()
    NDCG /= cnt
    return NDCG

In [8]:
tr_pos, te_pos = parser.data
tr_ds = Dataset(len(parser.user_label))
tr_loader = DataLoader(tr_ds, batch_size = config["batch_size"], shuffle = True, drop_last = False)

In [9]:
model = AutoRec(n_item = len(parser.item_label),
                n_feature = config["n_feature"]).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = config["lr"])
criterion = nn.CrossEntropyLoss()
#criterion = nn.MSELoss()

# 6. Training

In [10]:
best = 0
for ep in range(config["epoch"]):
    tr_loss = train(model, tr_loader, criterion, optimizer, parser,
                    prefix = "epoch: {0:03d}/{1:03d}".format(ep + 1, config["epoch"]), leave = False)

    ndcg_score = evaluate(model, tr_loader, parser, k = config["top_k"],
                          prefix = "epoch: {0:03d}/{1:03d} | tr_loss: {2:.4e}".format(ep + 1, config["epoch"], tr_loss))

    if best < ndcg_score:
        best = ndcg_score
        torch.save(model.state_dict(), config["model_path"])

epoch: 001/030 | tr_loss: 7.0024e+00 | ndcg: 0.0186: 100%|██████████| 188/188 [00:39<00:00,  4.77it/s]
epoch: 002/030 | tr_loss: 6.9248e+00 | ndcg: 0.0187: 100%|██████████| 188/188 [00:39<00:00,  4.76it/s]
epoch: 003/030 | tr_loss: 6.9196e+00 | ndcg: 0.0186: 100%|██████████| 188/188 [00:39<00:00,  4.75it/s]
epoch: 004/030 | tr_loss: 6.9158e+00 | ndcg: 0.0187: 100%|██████████| 188/188 [00:39<00:00,  4.73it/s]
epoch: 005/030 | tr_loss: 6.9102e+00 | ndcg: 0.0188: 100%|██████████| 188/188 [00:39<00:00,  4.76it/s]
epoch: 006/030 | tr_loss: 6.9015e+00 | ndcg: 0.0188: 100%|██████████| 188/188 [00:39<00:00,  4.74it/s]
epoch: 007/030 | tr_loss: 6.8897e+00 | ndcg: 0.0190: 100%|██████████| 188/188 [00:39<00:00,  4.73it/s]
epoch: 008/030 | tr_loss: 6.8764e+00 | ndcg: 0.0192: 100%|██████████| 188/188 [00:40<00:00,  4.67it/s]
epoch: 009/030 | tr_loss: 6.8613e+00 | ndcg: 0.0194: 100%|██████████| 188/188 [00:39<00:00,  4.73it/s]
epoch: 010/030 | tr_loss: 6.8438e+00 | ndcg: 0.0201: 100%|██████████| 188

# 7. Recommend by Best Model
- 상위 50개 제품에 대해 추천 진행(50개 미만 추천은 오류 처리)
- csv 확장자로 저장

In [11]:
print(best)

0.02247829314019395


In [12]:
def recommend(model, data_loader, parser, k = 50, leave = True): #학습된 모델을 사용하여 추천을 수행
    model.eval()
    pred = {}

    tr_pos, te_pos = parser.data

    with torch.no_grad():
        for i, users in enumerate(tqdm(data_loader, total = len(data_loader), leave = leave, desc = "recommend")):
            mat = parser.to_matrix(tr_pos, users)
            mat = mat.to(device)

            recon_mat = model(mat)
            recon_mat = recon_mat.softmax(dim = 1)
            recon_mat[mat == 1] = -1.
            rec_list = recon_mat.argsort(dim = 1)

            for user, rec in zip(users, rec_list):
                up = rec[-k:].cpu().numpy()[::-1].tolist()
                pred[user.item()] = up

    submit = pd.DataFrame(columns=["user_id", "item_id"])
    user_list, item_list = [], []
    for user, rec in tqdm(pred.items()):
        user = parser.user_decoder[user]
        for item in rec:
            item = parser.item_decoder[item]
            user_list.append(int(user))
            item_list.append(int(item))
    submit["user_id"] = user_list
    submit["item_id"] = item_list
    return submit

In [13]:
model.load_state_dict(torch.load(config["model_path"]))

<All keys matched successfully>

In [14]:
submission = recommend(model, tr_loader, parser)

100%|██████████| 192403/192403 [00:04<00:00, 47732.68it/s]


In [15]:
submission.to_csv(config["submit_path"], index=False)
print("Done.")

Done.
